In [1]:
import cv2
import numpy as np
import os
from PIL import Image,ImageTk
import tkinter as tk
from tkinter import messagebox
import mysql.connector

## Generating dataset

In [2]:
import cv2
print(cv2.data.haarcascades)

c:\Windows\System32\tensorflow-env\lib\site-packages\cv2\data\


In [ ]:
def gen_db():
    face_clf = cv2.CascadeClassifier(r"C:\Windows\System32\tensorflow-env\Lib\site-packages\cv2\data\haarcascade_frontalface_alt2.xml")


    # Extracting images from camera
    def face_data(image):
        image_gry =  cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
        face = face_clf.detectMultiScale(image_gry,1.3,5)
        # setting scaling factor as 1.3 and minimum Neighbour as  5

        if len(face) == 0:
            return None
            
        if face_data is None:
            return None
        for (x,y,w,h)  in face:
            sample = image[y:y+h,x:x+w]
        return sample

    vcap = cv2.VideoCapture(0)
    user = 1
    user_images = 0

    # Opening the camera
    while True:
        r,frame = vcap.read()
        if face_data(frame) is not None:
            user_images = user_images + 1
            detected_face = cv2.resize(face_data(frame), (200,200))
            detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)
            file_name_path = "dataset/user."+ str(user) + "." + str(user_images) + "." + "jpg"
            cv2.imwrite(file_name_path,detected_face)

            # displaying the text 
            cv2.putText(detected_face,str(user_images),(50,50),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,20),2)
            cv2.imshow("Detected Face", detected_face)

            # break condition 
            if (cv2.waitKey(1) == ord('x') & 0xff) or  int(user_images)== 200:
                print("Exiting ...")
                break
    vcap.release()
    cv2.destroyAllWindows()
    print("Database of the User is generated....")


## Training the Classifier

In [5]:
def clf(folder):
    path = [os.path.join(folder,img_path)
            for img_path in os.listdir(folder)
           if img_path.lower().endswith(('.png', '.jpg', '.jpeg'))]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imgArr = np.array(img,'uint8')
        id = int(os.path.split(image)[1].split(".")[1])


        faces.append(imgArr)
        ids.append(id)

    ids = np.array(ids)
    # Train and save classifier
    clsf = cv2.face.LBPHFaceRecognizer_create()
    clsf.train(faces,ids)
    clsf.write("classifier.xml")
# clf('dataset')

## Detecting the Face and Draw a Boundary

## Recognizing the Face

In [6]:
def boundBox(img, classifier, scF, minN, color, txt, clf):
    img_gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_ft = classifier.detectMultiScale(img_gry, scF, minN)
    coords = []

    for (x, y, w, h) in img_ft:
        # Predict the face region
        face = img_gry[y:y + h, x:x + w]
        id, pred = clf.predict(face)

        # Calculate confidence
        confidence = int(100 * (1 - pred / 300))

        # Determine label or unknown
        if confidence > 77:
            if id == 2:
                label = "Upanshu"
            else:
                label = "UNKNOWN"
            cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, color, 1, cv2.LINE_8)

        # Draw bounding box
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        coords = [x, y, w, h]

    return coords

def rcz(img, clf, faceCascade):
    coords = boundBox(img, faceCascade, 1.1, 10, (0, 255, 0), "Face", clf)
    return img

# Load face cascade and trained classifier
faceCascade = cv2.CascadeClassifier(
    r"C:\Windows\System32\tensorflow-env\Lib\site-packages\cv2\data\haarcascade_frontalface_alt2.xml"
)
clsf = cv2.face.LBPHFaceRecognizer_create()
clsf.read('classifier.xml')

# Open the video capture
vcap = cv2.VideoCapture(0)

while True:
    r, img = vcap.read()
    if not r:
        print("Failed to capture video")
        break

    img = rcz(img, clsf, faceCascade)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

vcap.release()
cv2.destroyAllWindows()

## Making the GUI

In [7]:
window = tk.Tk()
window.title("Face Recognition system")

l1 = tk.Label(window, text="NAME", font=("Helvetica",20))

l1.grid(column=0, row=0)

t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)
 

def gen_db():
    if t1.get() == "":
        messagebox.showinfo('Result', 'Please provide the Name')
    else:
        # Connect to the database
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            database="facedetct",
            use_pure=True
        )
        mycursor = mydb.cursor()
        mycursor.execute("SELECT MAX(id) FROM faces")
        myresult = mycursor.fetchall()
        id = 1 if myresult[0][0] is None else myresult[0][0] + 1

        # Insert new user
        sql = "INSERT INTO faces(id, Name) VALUES (%s, %s)"
        val = (id, t1.get())
        mycursor.execute(sql, val)
        mydb.commit()

        face_clf = cv2.CascadeClassifier(r"C:\Windows\System32\tensorflow-env\Lib\site-packages\cv2\data\haarcascade_frontalface_alt2.xml")

        # Function to extract face data
        def face_data(image):
            image_gry = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            face = face_clf.detectMultiScale(image_gry, 1.3, 5)
            if len(face) == 0:
                return None
            for (x, y, w, h) in face:
                sample = image[y:y + h, x:x + w]
            return sample

        # Capture images
        vcap = cv2.VideoCapture(0)
        user_images = 0

        while True:
            r, frame = vcap.read()
            if face_data(frame) is not None:
                user_images += 1
                detected_face = cv2.resize(face_data(frame), (200, 200))
                detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)

                # Use the unique user ID for file naming
                file_name_path = f"dataset/user.{id}.{user_images}.jpg"
                cv2.imwrite(file_name_path, detected_face)

                cv2.putText(detected_face, str(user_images), (50, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 255, 20), 2)
                cv2.imshow("Detected Face", detected_face)

                # Stop capturing after 200 images or if 'x' is pressed
                if cv2.waitKey(1) == ord('x') or user_images == 200:
                    print("Exiting ...")
                    break

        vcap.release()
        cv2.destroyAllWindows()
        print("Database of the User is generated....")
        messagebox.showinfo('Result', 'Database Generated...')



# b1 = tk.Button(window, text="Generate dataset", font=("Algerian",20), bg="pink", fg="black",command = gen_db)
b1 = tk.Button(
    window, 
    text="Generate dataset", 
    font=("Comic Sans MS", 10, "bold"), 
    bg="black", 
    fg="white", 
    activebackground="white", 
    activeforeground="black",
    width=20,   
    height=2,
    command=gen_db,
    bd=6,  
    relief="raised" 
)
b1.grid(column=0, row=4) 

def clf():
    folder = r'C:\Users\acer\Desktop\learn\Sem7\Major Project\dataset'
    path = [os.path.join(folder,img_path)
            for img_path in os.listdir(folder)
           if img_path.lower().endswith(('.png', '.jpg', '.jpeg'))]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imgArr = np.array(img,'uint8')
        id = int(os.path.split(image)[1].split(".")[1])


        faces.append(imgArr)
        ids.append(id)

    ids = np.array(ids)
    # Train and save classifier
    clsf = cv2.face.LBPHFaceRecognizer_create()
    clsf.train(faces,ids)
    clsf.write("classifier.xml")
    messagebox.showinfo('Result','Training Completed...')

# b2 = tk.Button(window, text="Train", font=("Algerian",20),bg="orange",fg="red",command = clf)
b2 = tk.Button(
    window, 
    text="Train", 
    font=("Comic Sans MS", 10, "bold"), 
    bg="black", 
    fg="white", 
    activebackground="white", 
    activeforeground="black", 
    command=clf,
    width=20,
    height=2,
    bd=6,  
    relief="raised" 
)


b2.grid(column=2, row=4)

def result():
    def boundBox(img, classifier, scF, minN, color, txt, clf):
        img_gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_ft = classifier.detectMultiScale(img_gry, scF, minN)
        coords = []
    
        for (x, y, w, h) in img_ft:
            # Predict the face region
            face = img_gry[y:y + h, x:x + w]
            id, pred = clf.predict(face)
    
            # Calculate confidence
            confidence = int(100 * (1 - pred / 300))

            mydb = mysql.connector.connect(
                host = "localhost",
                user = "root",
                passwd = "",
                database = "facedetct",
                use_pure=True
                
            )
            mycursor = mydb.cursor()
            mycursor.execute("SELECT name from faces where id ="+ str(id))
            result = mycursor.fetchall()
            if result:
                count = ''.join(result[0])  
            else:
                count = "Unknown"
            count = ''+ ''.join(count)
            
            # Determine label or unknown
            if confidence > 77:
                label = count
            else:
                label = "UNKNOWN"
            cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, color, 1, cv2.LINE_8)
    
            # Draw bounding box
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            coords = [x, y, w, h]
    
        return coords

    def rcz(img, clf, faceCascade):
        coords = boundBox(img, faceCascade, 1.1, 10, (0, 255, 0), "Face", clf)
        return img
    
    # Load face cascade and trained classifier
    faceCascade = cv2.CascadeClassifier(
        r"C:\Windows\System32\tensorflow-env\Lib\site-packages\cv2\data\haarcascade_frontalface_alt2.xml"
    )
    clsf = cv2.face.LBPHFaceRecognizer_create()
    clsf.read('classifier.xml')
    
    # Open the video capture
    vcap = cv2.VideoCapture(0)
    
    while True:
        r, img = vcap.read()
        if not r:
            print("Failed to capture video")
            break
    
        img = rcz(img, clsf, faceCascade)
        cv2.imshow("Face Detection", img)
    
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break
    
    vcap.release()
    cv2.destroyAllWindows()

b3 = tk.Button(
    window, 
    text="Detect the faces", 
    font=("Comic Sans MS", 10, "bold"), 
    bg="black", 
    fg="white", 
    activebackground="white", 
    activeforeground="black", 
    command=result,
    width=20,   
    height=2,
    bd=6,  
    relief="raised" 
)

b3.grid(column=1, row=4)
 # b3 = tk.Button(window, text="Detect the faces", font=("Algerian",20), bg="green", fg="orange",command = result)

b1.grid(column=0, row=4, padx=5, pady=5)
b2.grid(column=2, row=4, padx=5, pady=5)
b3.grid(column=1, row=4, padx=5, pady=5)

 
window.geometry("690x150")
window.mainloop()

## Integrating with MySQL

In [8]:
import mysql.connector
from mysql.connector import Error

try:
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        use_pure=True
    )
    if mydb.is_connected():
        print("MySQL connection successful!")
except Error as e:
    print(f"Error: {e}")


MySQL connection successful!


In [ ]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE faceDetct")

In [10]:
mycursor.execute("SHOW DATABASES")
for i in mycursor:
    print(i)

('facedetct',)
('information_schema',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('test',)


In [17]:
mydb.database = "faceDetct" 

In [21]:
mycursor.execute("USE faceDetct")
mycursor.execute("""
        CREATE TABLE IF NOT EXISTS faces (
            id INT PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(20)
        )
    """)
print("Table 'faces' created successfully!")

Table 'faces' created successfully!
